In [4]:
!nvidia-smi

Thu Oct  9 17:38:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 7.7 MB/s eta 0:00:00


In [6]:
!pip install -q transformers einops accelerate bitsandbytes langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.8 MB/s eta 0:00:00


In [7]:
!pip install sentence_transformers

In [8]:
!pip install llama-index==0.9.39

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00


## Create a Folder named "Data", and add your files inside

In [10]:
!mkdir Data

In [11]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

## Loading the PDF (knowledgebase we have)

In [12]:
documents = SimpleDirectoryReader("/content/Data").load_data()

In [ ]:
documents

[Document(id_='ebc6ed49-1968-4513-a60f-66a50f2c0d9e', embedding=None, metadata={'page_label': '1', 'file_name': 'stats.pdf', 'file_path': '/content/Data/stats.pdf', 'file_type': 'application/pdf', 'file_size': 51301, 'creation_date': '2024-06-10', 'last_modified_date': '2024-06-10', 'last_accessed_date': '2024-06-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0

In [14]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

Log in to huggingface with a token key if the model you need is Private

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

## Load the pretrained LLM model from open-source HuggingFace

In [ ]:
## Loading Mistral model

import torch

llm = HuggingFaceLLM(
    context_window = 4096,
    max_new_tokens = 256,
    generate_kwargs = {"temperature": 0.0, "do_sample": False},
    system_prompt = system_prompt,
    query_wrapper_prompt= query_wrapper_prompt,
    tokenizer_name = "mistralai/Mistral-7B-Instruct-v0.1",
    model_name = "mistralai/Mistral-7B-Instruct-v0.1",
    device_map = 'auto',
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

In [ ]:
# import torch

# llm = HuggingFaceLLM(
#     context_window = 4096,
#     max_new_tokens = 256,
#     generate_kwargs = {"temperature": 0.0, "do_sample": False},
#     system_prompt = system_prompt,
#     query_wrapper_prompt= query_wrapper_prompt,
#     tokenizer_name = "tiiuae/falcon-7b",
#     model_name = "tiiuae/falcon-7b",
#     device_map = 'auto',
#     # uncomment this if using CUDA to reduce memory usage
#     model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
# )

In [16]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [17]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

## We load the text embedder from HuggingFace for our PDF/knowledgebase

In [ ]:
embed_model = LangchainEmbedding(
    HuggingFaceBgeEmbeddings(model_name ="sentence-transformers/all-mpnet-base-v2")
)

## We connect the embder to our conversational LLM, and we set the chunk size

In [19]:
 service_context = ServiceContext.from_defaults(
    chunk_size = 1024,
    llm = llm,
    embed_model = embed_model
)

## We add the our PDF document to the knowledgebase which is connected to the embedder

In [20]:
index = VectorStoreIndex.from_documents(documents, service_context= service_context)

In [21]:
query_engine = index.as_query_engine()

In [24]:
query = "What are the problems occurring in the paper industry, which share similarities with many other production scheduling problems?"
response = query_engine.query(query)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The problems occurring in the paper industry, which share similarities with many other production scheduling problems, include sequence-dependent changeover times or costs, which are a common feature of paper production scheduling models. These costs depend on the starting and ending grades of the grade change and can affect the production process.
